## Compte rendu R5.A.10, ANDRIAMISA Nayann, Groupe Nayru

#### Les installations :

In [8]:
pip install redis

Note: you may need to restart the kernel to use updated packages.


In [6]:
import redis

# Créer une connexion à Redis
connection = redis.Redis(host='localhost', port=6379, decode_responses=True)

connection.set('foo', 'arabe salope')

print(connection.get('foo'))

arabe salope


In [ ]:
pip install pymongo

In [6]:
import pymongo

# Connexion à MongoDB
client = pymongo.MongoClient("mongodb://192.168.0.5:27017/")

print(client)

db_name = "test"
db = client[db_name]
print(db)


MongoClient(host=['192.168.0.5:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['192.168.0.5:27017'], document_class=dict, tz_aware=False, connect=True), 'test')
